In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

print('Loading data ...')

train = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\train.csv")
prop = pd.read_csv("final_prop_encoded_onehot.csv")

print('done')

Loading data ...
done


In [2]:
prop.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,fullbathcnt,garagecarcnt,...,propertylandusetypeid_264.0,propertylandusetypeid_265.0,propertylandusetypeid_266.0,propertylandusetypeid_267.0,propertylandusetypeid_269.0,propertylandusetypeid_270.0,propertylandusetypeid_275.0,propertylandusetypeid_279.0,propertylandusetypeid_31.0,propertylandusetypeid_47.0
0,10754147,0.0,0.0,5.0,3652.0,3652.0,6037.0,0.0,5.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,10759547,0.0,0.0,1.0,800.0,800.0,6037.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10843547,0.0,0.0,1.0,73026.0,1274.0,6037.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,10859147,0.0,0.0,2.0,5068.0,1041.0,6037.0,0.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,10879947,0.0,0.0,1.0,1776.0,784.0,6037.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
# print(prop.shape)
# prop['airconditioningtypeid'] = prop['airconditioningtypeid'].astype(str)
# prop['buildingqualitytypeid'] = prop['buildingqualitytypeid'].astype(str)
# prop['heatingorsystemtypeid'] = prop['heatingorsystemtypeid'].astype(str)
# prop['pooltypeid7'] = prop['pooltypeid7'].astype(str)
# prop['propertylandusetypeid'] = prop['propertylandusetypeid'].astype(str)
# cat = ['airconditioningtypeid','buildingqualitytypeid','heatingorsystemtypeid','pooltypeid7','propertylandusetypeid']
# prefixes = {
#     'airconditioningtypeid':'airconditioningtypeid',
#     'buildingqualitytypeid':'buildingqualitytypeid',
#     'heatingorsystemtypeid':'heatingorsystemtypeid',
#     'pooltypeid7':'pooltypeid7',
#     'propertylandusetypeid':'propertylandusetypeid'
# }
# prop = pd.get_dummies(prop, prefix=prefixes, columns=cat)
# print(prop.shape)

In [4]:
train.head()

,parcelid,logerror,transactiondate,subdate
0,11016594,0.0276,1/1/2016,201601
1,14366692,-0.1684,1/1/2016,201601
2,12098116,-0.0040,1/1/2016,201601
3,12643413,0.0218,2/1/2016,201601
4,14432541,-0.0050,2/1/2016,201601


In [5]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'subdate'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

split = 110000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression_l1'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 80
params['min_data'] = 300
params['min_hessian'] = 0.1

watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

print("Prepare for the prediction ...")
sample = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\sample_submission.csv")
sample.columns = ['parcelid','201704','201705','201706','201707','201708','201709']
df_test = sample.merge(prop, on='parcelid', how='left')
del prop; gc.collect()
x_test = df_test[train_columns]
del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

print("Start prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)

del x_test; gc.collect()

print("Start write result ...")

for c in sample.columns[sample.columns != 'parcelid']:
    sample[c] = p_test

submission = sample
del sample

(113039, 70) (113039,)
[1]	valid_0's l1: 0.0730985
[2]	valid_0's l1: 0.0730934
[3]	valid_0's l1: 0.0730885
[4]	valid_0's l1: 0.0730836
[5]	valid_0's l1: 0.0730789
[6]	valid_0's l1: 0.0730744
[7]	valid_0's l1: 0.0730695
[8]	valid_0's l1: 0.0730653
[9]	valid_0's l1: 0.0730606
[10]	valid_0's l1: 0.073056
[11]	valid_0's l1: 0.0730514
[12]	valid_0's l1: 0.0730472
[13]	valid_0's l1: 0.0730428
[14]	valid_0's l1: 0.073038
[15]	valid_0's l1: 0.0730334
[16]	valid_0's l1: 0.0730291
[17]	valid_0's l1: 0.0730248
[18]	valid_0's l1: 0.0730202
[19]	valid_0's l1: 0.0730158
[20]	valid_0's l1: 0.0730116
[21]	valid_0's l1: 0.073007
[22]	valid_0's l1: 0.0730023
[23]	valid_0's l1: 0.0729976
[24]	valid_0's l1: 0.0729929
[25]	valid_0's l1: 0.0729883
[26]	valid_0's l1: 0.0729839
[27]	valid_0's l1: 0.0729794
[28]	valid_0's l1: 0.0729752
[29]	valid_0's l1: 0.072971
[30]	valid_0's l1: 0.0729665
[31]	valid_0's l1: 0.0729622
[32]	valid_0's l1: 0.0729581
[33]	valid_0's l1: 0.0729535
[34]	valid_0's l1: 0.0729493
[35]

[278]	valid_0's l1: 0.0723
[279]	valid_0's l1: 0.0722985
[280]	valid_0's l1: 0.0722969
[281]	valid_0's l1: 0.0722954
[282]	valid_0's l1: 0.0722938
[283]	valid_0's l1: 0.0722923
[284]	valid_0's l1: 0.0722905
[285]	valid_0's l1: 0.0722891
[286]	valid_0's l1: 0.0722874
[287]	valid_0's l1: 0.072286
[288]	valid_0's l1: 0.0722845
[289]	valid_0's l1: 0.0722829
[290]	valid_0's l1: 0.0722815
[291]	valid_0's l1: 0.07228
[292]	valid_0's l1: 0.0722783
[293]	valid_0's l1: 0.0722769
[294]	valid_0's l1: 0.0722755
[295]	valid_0's l1: 0.072274
[296]	valid_0's l1: 0.0722724
[297]	valid_0's l1: 0.072271
[298]	valid_0's l1: 0.0722696
[299]	valid_0's l1: 0.0722679
[300]	valid_0's l1: 0.0722665
[301]	valid_0's l1: 0.0722651
[302]	valid_0's l1: 0.0722636
[303]	valid_0's l1: 0.0722622
[304]	valid_0's l1: 0.0722605
[305]	valid_0's l1: 0.0722591
[306]	valid_0's l1: 0.0722575
[307]	valid_0's l1: 0.0722559
[308]	valid_0's l1: 0.0722544
[309]	valid_0's l1: 0.0722533
[310]	valid_0's l1: 0.072252
[311]	valid_0's l1:

In [6]:
from math import fabs as absolute
test_path = "C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\test.csv"
test = pd.read_csv(test_path, low_memory=False)
test_size=len(test)

In [7]:
total_ae = 0.0
print("Processing ", test_size, "rows")
for index,row in test.iterrows():
    r = submission.loc[submission['parcelid'] == row[0]]
    
    pred_error = r[str(row[3])]
    total_ae += absolute(pred_error - row[1])
    
    if index % 10000 == 0 and index !=0:
        print("Completed: ", index, "rows")
        
print("===Process Completed===")
print("\n Final score:", total_ae/test_size)

Processing  54849 rows
Completed:  10000 rows
Completed:  20000 rows
Completed:  30000 rows
Completed:  40000 rows
Completed:  50000 rows
===Process Completed===

 Final score: 0.06872114197890064


7